In [2]:
import sys
sys.path
sys.path.append("/home/haoqi.whq/llm-inference/LoRA")

In [3]:
from src.model import GPT2Config, GPT2LMModel
import torch
from loralib import PruneLayer
import loralib as lora
import copy
import importlib
from torchsummary import summary


## load model

In [67]:
state_dict = torch.load("./pretrained_checkpoints/gpt2-medium-pytorch_model.bin")
# print(state_dict)
def split_attention_weights(state_dict):
    state_dict_tmp = copy.deepcopy(state_dict)
    attn_lists = ["q_attn", "k_attn", "v_attn"]
    for n, p in state_dict.items():
        if "c_attn" in n:
            module, name = n.split("c_attn.")
            dim = 0 if name == "bias" else 1
            split_size = p.shape[0] if name == "weight" else p.shape[0] // 3
            params = p.split(split_size, dim=dim)
            for i, attn in enumerate(attn_lists):
                # print(module+attn+"."+name)
                state_dict_tmp[module+attn+"."+name] = params[i]
    return state_dict_tmp

state_dict_tmp = split_attention_weights(state_dict)
print(state_dict_tmp["h.0.attn.q_attn.weight"].shape)

torch.Size([1024, 1024])


## Load sample data

In [68]:
from torch.utils.data import DataLoader
from src.data_utils import FT_Dataset

train_data = "./data/e2e/train.jsonl"
train_batch_size = 2
seq_len = 512
obj = "clm"
label_smooth = 0.1
train_data = FT_Dataset(
    train_data, train_batch_size, seq_len, joint_lm= obj == "jlm"
)

train_loader = DataLoader(
    train_data,
    batch_size=train_batch_size,
    num_workers=0,
    shuffle=False,
    pin_memory=False,
    drop_last=True,
    # sampler=torch(
    #     train_data, seed=10
    # ),
)
print(type(train_loader))
# data = train_loader[0]

for i, data in enumerate(train_loader):
    data = {key: value for key, value in data.items()}
    _input = data["input"]
    _target = data["target"]
    _msk = data["mask"]

    print(_input.shape)
    print(_target.shape)
    print(_msk.shape)
    break

<class 'torch.utils.data.dataloader.DataLoader'>
torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 512])


In [69]:
# medium GPT2
config = GPT2Config(
    n_embd=1024,
    n_layer=24,
    n_head=16,
    lora_attn_dim=4,
    lora_attn_alpha=32,
    lora_dropout=0.1,
    enable_mlp=True,
    enable_wo=True,
    enable_wq=True,
    enable_wk=True,
    enable_wv=True,
)

lm_net = GPT2LMModel(config)

# summary(lm_net, _input, lm_labels=_target, lm_mask=_msk, label_smooth=label_smooth, depth=6)
# print(lm_net.state_dict())
# transformer.h.0.attn.q_atten.lora_B
# lm_head.decoder.weight
# print("loading model pretrained weight.")
lm_net.load_lora_weight("./pretrained_checkpoints/gpt2-medium-pytorch_model.bin", torch.load("/home/haoqi.whq/llm-inference/LoRA/examples/NLG/trained_models/GPT2_M/e2e/model.total.26290.pt"))


O Attention using LoRA: PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
)
MLP using LoRA: PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
), PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
)


AttributeError: 'dict' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [ ]:
lm_net.load_weight(torch.load("/home/haoqi.whq/llm-inference/LoRA/examples/NLG/trained_models/GPT2_M/e2e/model.lora.1000.pt"))
for m in lm_net.modules():
    if isinstance(m, PruneLayer):
        print(m, m.complexity())

PruneLinear(
  in_features=1024, out_features=1024, bias=True
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(64219.3633, grad_fn=<MulBackward0>)
PruneLinear(
  in_features=1024, out_features=1024, bias=True
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(64219.4727, grad_fn=<MulBackward0>)
PruneLinear(
  in_features=1024, out_features=1024, bias=True
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(64217.6719, grad_fn=<MulBackward0>)
PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(64218.4102, grad_fn=<MulBackward0>)
PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(160547.5781, grad_fn=<MulBackward0>)
PruneGPTConv1D(
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(160545.0938, grad_fn=<MulBackward0>)
PruneLinear(
  in_features=1024, out_features=1024, bias=True
  (lora_dropout): Dropout(p=0.1, inplace=False)
) tensor(64219.2461, grad_fn=<MulBackward0>)
PruneLinear(
  in_features=1024, out_features=1024,

In [ ]:
torch.load("/home/haoqi.whq/llm-inference/LoRA/examples/NLG/trained_models/GPT2_M/e2e/model.lora.10000.pt")

{'model_state_dict': {'module.module.transformer.h.0.attn.q_attn.lora_A': tensor([[-0.0025,  0.0516, -0.0029,  ...,  0.0111, -0.0188,  0.0219],
          [-0.0106,  0.0413,  0.0110,  ..., -0.0542, -0.0071,  0.0514],
          [-0.0188,  0.0177, -0.0030,  ...,  0.0506, -0.0002,  0.0035],
          [ 0.0025,  0.0364,  0.0307,  ..., -0.0258,  0.0084,  0.0512]],
         device='cuda:0'),
  'module.module.transformer.h.0.attn.q_attn.lora_B': tensor([[-1.6337e-02,  7.5459e-03, -1.6362e-02, -4.9124e-03],
          [-3.9538e-03, -1.3736e-02, -1.0740e-02, -1.4239e-02],
          [ 5.1056e-03, -7.3317e-03,  2.0945e-03, -4.9228e-03],
          ...,
          [-1.6659e-02,  1.3963e-02, -2.3729e-02, -4.8820e-03],
          [ 8.8498e-03, -1.6449e-02,  1.3057e-02, -5.8411e-05],
          [-3.6035e-03, -1.2772e-02, -1.2854e-02,  1.6703e-03]], device='cuda:0'),
  'module.module.transformer.h.0.attn.q_attn.lora_scaling': tensor(6.2918, device='cuda:0'),
  'module.module.transformer.h.0.attn.k_attn.lora

In [ ]:
lora.mark_only_lora_as_trainable(lm_net)
for m in lm_net.modules():
    if isinstance(m, PruneLayer) and hasattr(m, "lora_scaling"):
        print(m.lora_scaling)

for n, p in lm_net.named_parameters():
    if "lora_" in n:
        print(n)
# for n, p in lm_net.named_parameters():
#     if "lora" in n:
#         print(n)

Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.1977, requires_grad=True)
Parameter containing:
tensor(5.

## Efficiency Comparison
The baseline is all-in-tee.
The usage of lora in tee is to reduce the computation complexity in tee, consequently the runtime.

qkv statistics
- GPU linear time: 0.1957416534423828 ms
- CPU linear time: 9.22703742980957 ms
- CPU lora time: 1.2764930725097656 ms

mlp statistics
- GPU linear time: 0.3578662872314453 ms
- CPU linear time: 69.11993026733398 ms
- CPU lora time: 5.603551864624023 ms

In [4]:
import time
import math
import torch

B = 8
SEQ_LEN = 512
r = 32

# [base, medium, large]
MODEL = "medium"

if MODEL == "base":
    n_embd = 768
    n_head = 12
elif MODEL == "medium":
    n_embd = 1024
    n_head = 16
elif MODEL == "large":
    n_embd = 1280
    n_head = 20
else:
    raise NotImplementedError(f"Model {MODEL} not supported.")

# GPU linear time: 0.19550323486328125 ms
# CPU linear time: 9.151220321655273 ms
# CPU lora time: 1.3489723205566406 ms
def attention_qkv():
    x = torch.rand(size=(B, SEQ_LEN, n_embd))
    
    # GPU computations
    start = time.time()
    x = x.cuda()
    print(f"[qkv] CPU -> GPU io time: {(time.time()-start)*1000} ms | {x.dtype} | {x.shape}")
    
    gpu_linear = torch.nn.Linear(n_embd, n_embd).cuda()

    start = time.time()
    gpu_res = gpu_linear(x)
    end = time.time()
    print(f"[qkv] GPU linear time: {(end-start)*1000} ms")

    x = x.cpu()
    cpu_linear = gpu_linear.cpu()
    start = time.time()
    cpu_res = cpu_linear(x)
    end = time.time()
    print(f"CPU linear time: {(end-start)*1000} ms")

    # cpu lora computations
    x = x.cpu()
    lora_A = torch.nn.Parameter(torch.rand((r, n_embd)))
    lora_B = torch.nn.Parameter(torch.rand((n_embd, r)))
    start = time.time()
    cpu_lora = x @ lora_A.transpose(0, 1) @ lora_B.transpose(0, 1)
    end = time.time()
    print(f"CPU lora time: {(end-start)*1000} ms")

    start = time.time()
    gpu_res = gpu_res.cpu()
    end = time.time()
    print(f"[qkv] GPU -> CPU io time: {(end-start)*1000} ms | {gpu_res.dtype} | {gpu_res.shape}")

# Same as qkv
def attention_o():
    x = torch.rand(size=(B, SEQ_LEN, n_embd))
    
    # GPU computations
    x = x.cuda()
    gpu_linear = torch.nn.Linear(n_embd, n_embd).cuda()
    start = time.time()
    gpu_res = gpu_linear(x)
    end = time.time()

    x = x.cpu()
    cpu_linear = gpu_linear.cpu()
    start = time.time()
    cpu_res = cpu_linear(x)
    end = time.time()
    print(f"CPU linear time: {(end-start)*1000} ms")

    # cpu lora computations
    x = x.cpu()
    lora_A = torch.nn.Parameter(torch.rand((r, n_embd)))
    lora_B = torch.nn.Parameter(torch.rand((n_embd, r)))
    start = time.time()
    cpu_lora = x @ lora_A.transpose(0, 1) @ lora_B.transpose(0, 1)
    end = time.time()
    print(f"CPU lora time: {(end-start)*1000} ms")

def gelu(x):
    return (
        0.5
        * x
        * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
    )
def gelu_impl(x):
    """OpenAI's gelu implementation."""
    return (
        0.5 * x * (1.0 + torch.tanh(0.7978845608028654 * x * (1.0 + 0.044715 * x * x)))
    )

def attention_mlp():
    x = torch.rand(size=(B, SEQ_LEN, n_embd))
    
    # GPU computations
    start = time.time()
    # x = x.to(torch.int8)
    x = x.cuda()
    end = time.time()
    print(f"[FC1] CPU -> GPU io time: {(end - start)*1000} ms | {x.dtype} | {x.shape}")

    gpu_linear1 = torch.nn.Linear(n_embd, n_embd*4).cuda()
    gpu_linear2 = torch.nn.Linear(n_embd*4, n_embd).cuda()

    act = gelu_impl

    start = time.time()
    h1 = gpu_linear1(x)
    end = time.time()
    print(f"[FC1] GPU time {(end - start) * 1000} ms")

    print(f"{'='*10}  TEST START {'='*10}")

    start = time.time()
    h1 = act(h1)
    print(h1.shape)
    end = time.time()
    print(f"[ACT] GPU time {(end - start) * 1000} ms")
    print(f"{'='*10}  TEST END {'='*10}")

    # sparsity ops
    # h1 = h1.to(torch.int8)
    topk, indices = torch.topk(h1, k = 500, dim=-1)
    print(topk.shape)
    h1 = torch.zeros_like(h1).scatter(-1, indices, topk)
    
    start = time.time()
    h1 = h1.view(-1, h1.size(-1))
    h1_sparse = h1.to_sparse_csr()

    h1_sparse = h1_sparse.cpu()
    # h1 = h1.cpu()
    end = time.time()
    print(f"[FC1] GPU -> CPU io time: {(end-start)*1000} ms | {h1.dtype} | {h1.shape}")

    print(f"{'='*10}  TEST START {'='*10}")
    
    start = time.time()
    act_values = gelu(h1_sparse.values())
    print(f"[FC1] CPU sparse gelu time: {(time.time()-start)*1000} ms | {h1_sparse.dtype} | {h1_sparse.shape}")

    start = time.time()
    # print(h1_sparse)
    # print(act_values)
    # 1. Sparse Csr not have strides
    # h1_sparse.values().copy_(h1_sparse.values())
    # 2.
    # h1_sparse.values().copy_(act_values.data)
    # 3. work
    modified_sparse_tensor = torch.sparse_csr_tensor(
        h1_sparse.crow_indices(),
        h1_sparse.col_indices(),
        act_values,
        h1_sparse.size(),
        dtype=h1_sparse.dtype,
        device=h1_sparse.device,
    )
    # print(modified_sparse_tensor)
    print(f"[FC1] CPU assign new act values time: {(time.time()-start)*1000} ms | {h1.dtype} | {h1.shape}")

    print(f"{'='*10}  TEST END {'='*10}")

    start = time.time()
    h1 = h1_sparse.to_dense()
    print(f"[FC1] GPU -> CPU to dense time: {(time.time()-start)*1000} ms | {h1.dtype} | {h1.shape}")


    start = time.time()
    h1 = act(h1)
    end = time.time()
    print(f"[ACT] cpu time {(end - start) * 1000} ms")

    # h1 = h1.half()    
    start = time.time()
    h1 = h1.cuda()
    end = time.time()
    print(f"[FC2] CPU -> GPU io time: {(end-start)*1000} ms | {h1.dtype} | {h1.shape}")

    start = time.time()
    gpu_res = gpu_linear2(h1)
    end = time.time()
    print(f"[FC2] GPU time: {(end-start)*1000} ms")

    print(f"{'='*10}  TEST START {'='*10}")
    start = time.time()
    h1_sparse = modified_sparse_tensor.cuda()
    end = time.time()
    print(f"[FC2] CPU -> GPU sparse io time: {(end-start)*1000} ms | {h1.dtype} | {h1.shape}")

    start = time.time()
    gpu_res = torch.sparse.mm(h1_sparse, gpu_linear2.weight.t())
    end = time.time()
    print(f"[FC2] GPU sparse linear time: {(end-start)*1000} ms")

    print(f"{'='*10}  TEST END {'='*10}")


    start = time.time()
    # gpu_res = gpu_res.to(torch.int8)
    gpu_res = gpu_res.cpu()
    end = time.time()
    print(f"[FC2] GPU -> CPU io time: {(end-start)*1000} ms | {gpu_res.dtype} | {gpu_res.shape}")

    x = x.cpu()
    cpu_linear1 = gpu_linear1.cpu()
    cpu_linear2 = gpu_linear2.cpu()
    start = time.time()
    cpu_res = cpu_linear2(cpu_linear1(x))
    end = time.time()
    print(f"CPU linear time: {(end-start)*1000} ms")

    # cpu lora computations
    x = x.cpu()
    lora_A = torch.nn.Parameter(torch.rand((r, n_embd)))
    lora_B = torch.nn.Parameter(torch.rand((n_embd*4, r)))

    lora_A2 = torch.nn.Parameter(torch.rand((r, n_embd*4)))
    lora_B2 = torch.nn.Parameter(torch.rand((n_embd, r)))
    start = time.time()
    cpu_lora = x @ lora_A.transpose(0, 1) @ lora_B.transpose(0, 1)
    cpu_lora_res = cpu_lora @ lora_A2.transpose(0, 1) @ lora_B2.transpose(0, 1)
    end = time.time()
    print(f"CPU lora time: {(end-start)*1000} ms")

print(f"qkv statistics")
attention_qkv()

print(f"\nmlp statistics")
attention_mlp()

qkv statistics
[qkv] CPU -> GPU io time: 3.1425952911376953 ms | torch.float32 | torch.Size([8, 512, 1024])
[qkv] GPU linear time: 0.20456314086914062 ms
CPU linear time: 9.17673110961914 ms
CPU lora time: 1.3921260833740234 ms
[qkv] GPU -> CPU io time: 2.831697463989258 ms | torch.float32 | torch.Size([8, 512, 1024])

mlp statistics
[FC1] CPU -> GPU io time: 2.9022693634033203 ms | torch.float32 | torch.Size([8, 512, 1024])
[FC1] GPU time 0.21147727966308594 ms
==========  TEST START ==========
torch.Size([8, 512, 4096])
[ACT] GPU time 0.20956993103027344 ms
==========  TEST END ==========
torch.Size([8, 512, 500])
[FC1] GPU -> CPU io time: 12.537002563476562 ms | torch.float32 | torch.Size([4096, 4096])
==========  TEST START ==========
[FC1] CPU sparse gelu time: 1.8482208251953125 ms | torch.float32 | torch.Size([4096, 4096])
[FC1] CPU assign new act values time: 0.10967254638671875 ms | torch.float32 | torch.Size([4096, 4096])
==========  TEST END ==========
[FC1] GPU -> CPU to de

## Sparsity Analysis

In [ ]:
import time
import torch

sparsity_ratio = 0.99999

linear = torch.nn.Linear(1024, 3072, bias=False).float().cpu().eval()

sort_res = torch.sort(linear.weight, dim=-1, stable=True)
indices = sort_res[1][:, :int(linear.weight.shape[1]* sparsity_ratio)]
mask = (torch.zeros_like(linear.weight) == 1)
mask.scatter_(1, indices, True)
linear.weight.data[mask] = 0

x = torch.rand(3072, 1024).float().cpu()

with torch.inference_mode():
    start = time.time()
    # dense_output = linear(x)
    dense_output = torch.mm(linear.weight, x.t()).t()
    print(f"Dense linear {(time.time() - start) * 1000} ms")
    
    # convert to sparse format
    # linear.weight = torch.nn.Parameter(linear.weight.to_sparse_csr())

    indices = torch.tensor([[0, 1, 2],
                        [2, 0, 1]], dtype=torch.long)
    values = torch.tensor([1.0, 2.0, 3.0], dtype=torch.float)
    shape = torch.Size([3072, 1024])
    sparse_matrix = torch.sparse.FloatTensor(indices, values, shape)
    
    start = time.time()
    sparse_output = torch.sparse.mm(sparse_matrix, x.t()).t()
    print(f"Sparse test linear {(time.time() - start) * 1000} ms")


    dense_matrix_test = sparse_matrix.to_dense()
    # print(sparse_matrix)
    # print(dense_matrix_test)

    mat = linear.weight
    # linear.weight = dense_matrix_test
    # sparse_weight = torch.sparse.FloatTensor(mat.nonzero().t(), mat[mat != 0], mat.size())
    sparse_weight = linear.weight.to_sparse_csr()
    print(sparse_weight)
    start = time.time()
    sparse_output = torch.sparse.mm(sparse_weight, x.t()).t()
    # sparse_output = linear(x)
    print(f"Sparse linear {(time.time() - start) * 1000} ms")

    # sparse and dense matmul are numerically equivalent
    assert torch.allclose(sparse_output, dense_output, atol=1e-3)

Dense linear 21.256208419799805 ms
Sparse test linear 7.407903671264648 ms
tensor(crow_indices=tensor([   0,    1,    2,  ..., 3070, 3071, 3072]),
       col_indices=tensor([863, 417, 463,  ..., 409, 666, 350]),
       values=tensor([0.0312, 0.0311, 0.0312,  ..., 0.0310, 0.0311, 0.0312]),
       size=(3072, 1024), nnz=3072, layout=torch.sparse_csr)
Sparse linear 21.317720413208008 ms


In [ ]:
import torch
import time

W = 4
H = 4
# 创建稀疏矩阵
indices = torch.tensor([[0, 1, 2],
                        [2, 0, 1]], dtype=torch.long)
values = torch.tensor([1.0, 2.0, 3.0], dtype=torch.float)
shape = torch.Size([W, H])
sparse_matrix = torch.sparse.FloatTensor(indices, values, shape)

# 创建稠密矩阵
dense_matrix = torch.randn(H, W)

# torch.sparse.mm 进行稀疏矩阵乘法
start_time_sparse = time.time()
sparse_result = torch.sparse.mm(sparse_matrix, dense_matrix)
end_time_sparse = time.time()

# torch.mm 进行普通矩阵乘法
start_time_dense = time.time()
dense_result = torch.mm(sparse_matrix.to_dense(), dense_matrix)
end_time_dense = time.time()

# 计算时间差
time_taken_sparse = end_time_sparse - start_time_sparse
time_taken_dense = end_time_dense - start_time_dense

print("稀疏矩阵乘法的运行时间：", time_taken_sparse * 1000)
print("普通矩阵乘法的运行时间：", time_taken_dense * 1000)
print("稀疏矩阵乘法相对于普通矩阵乘法的速度提升：", time_taken_dense / time_taken_sparse)

稀疏矩阵乘法的运行时间： 0.18835067749023438
普通矩阵乘法的运行时间： 0.1671314239501953
稀疏矩阵乘法相对于普通矩阵乘法的速度提升： 0.8873417721518987
